In [200]:
from pathlib import Path
import re
import pandas as pd
import numpy as np

In [254]:
directory = '/home/zozi/Downloads/info-patients-v0_4/edf'

df = pd.DataFrame()

for path in Path(directory).rglob(f'*.txt'):
    sub_df = pd.DataFrame(columns=['Patient', 'Age'], index=[0])
    patient = path.parent.parent.name

    try:
        with open(path, 'r', encoding='utf-8') as f_open:
            for line in f_open:
                if line.startswith('CLINICAL HISTORY'):
                    sub_df['path'] = path
                    sub_df['Patient'] = path.parent.parent.name
                    r = re.search(r'(\d{1,2})[- ]', line)
                    if r:
                        sub_df['Age'] = r.group(1)
                    
                    r = re.search(r'woman|male|gentleman|female|man|toddler', line)
                    if r:
                        sub_df['Genre'] = r.group()
                    
                    df = df.append(sub_df)
                        
    except UnicodeDecodeError:
        with open(path, 'r', encoding='cp1252') as f_open:
            for line in f_open:
                if line.startswith('CLINICAL HISTORY'):
                    sub_df['path'] = path
                    sub_df['Patient'] = path.parent.parent.name
                    r = re.search(r'(\d{1,2})[- ]', line)
                    if r:
                        sub_df['Age'] = r.group(1)
                    
                    r = re.search(r'woman|male|gentleman|female|man|toddler', line)
                    if r:
                        sub_df['Genre'] = r.group()
                    
                    df = df.append(sub_df)

In [255]:
df.isnull().sum()

Patient      0
Age         72
path         0
Genre      101
dtype: int64

In [256]:
df.Age = df.Age.astype(float).groupby(df['Patient']).transform(lambda x: x.fillna(x.mean()))

In [257]:
df.Genre = df['Genre'].groupby(df['Patient']).transform(lambda x: x.fillna(method ='bfill'))

In [258]:
df.Genre = df['Genre'].groupby(df['Patient']).transform(lambda x: x.fillna(method ='ffill'))

In [259]:
df.isnull().sum()

Patient     0
Age        30
path        0
Genre      50
dtype: int64

In [260]:
with open('/home/zozi/Downloads/info-patients-v0_4/edf/train/02_tcp_le/007/00000752/s001_2003_05_16/00000752_s001.txt', 'r', encoding='cp1252') as f:
    for line in f:
        print(line)

CLINICAL HISTORY: This is a 59-year-old woman with hypertension, diabetes, afib, status-post cardiac arrest the previous day.



MEDICATIONS: Lisinopril, Amiodarone, Digoxin, others.



INTRODUCTION: Digital video EEG is performed at the bedside in the ICU using standard 10-20 system of electrode placement with one channel of EKG. The patient is intubated and comatose without any motor signs whatsoever. Stimulation of the patient is performed.



DESCRIPTION OF THE RECORD: The background EEG is markedly abnormal and does demonstrate variability.  The primary frequencies in the background are a mixture of theta and delta and there are some portions of the record that are richer than others in theta and.   There are frequent periodic epileptiform discharges identified in the frontotemporal regions bilaterally.  These are associated with theta frequency activity.  There is a somewhat poorly defined but ictal pattern identified in the record.  The pattern is one in which there is a theta f

In [261]:
df[df.Patient == '00000752'].path.tolist()

[PosixPath('/home/zozi/Downloads/info-patients-v0_4/edf/train/02_tcp_le/007/00000752/s001_2003_05_16/00000752_s001.txt')]

In [262]:
clean_df = df.drop_duplicates(subset=['Patient'], keep='first')

In [263]:
clean_df.isnull().sum()

Patient     0
Age        18
path        0
Genre      40
dtype: int64

In [264]:
clean_df.shape

(554, 4)

In [265]:
clean_df

,Patient,Age,path,Genre
0,00001027,50.0,/home/zozi/Downloads/info-patients-v0_4/edf/de...,male
0,00003635,6.0,/home/zozi/Downloads/info-patients-v0_4/edf/de...,male
0,00000795,27.0,/home/zozi/Downloads/info-patients-v0_4/edf/de...,male
0,00003306,13.0,/home/zozi/Downloads/info-patients-v0_4/edf/de...,female
0,00004671,22.0,/home/zozi/Downloads/info-patients-v0_4/edf/de...,gentleman
...,...,...,...,...
0,00008859,29.0,/home/zozi/Downloads/info-patients-v0_4/edf/tr...,woman
0,00008829,48.0,/home/zozi/Downloads/info-patients-v0_4/edf/tr...,female
0,00008835,21.0,/home/zozi/Downloads/info-patients-v0_4/edf/tr...,female
0,00008886,81.0,/home/zozi/Downloads/info-patients-v0_4/edf/tr...,female


In [266]:
clean_df[clean_df.Age.notna()].Age = clean_df[clean_df.Age.notna()].Age.astype(int)

In [267]:
clean_df

,Patient,Age,path,Genre
0,00001027,50.0,/home/zozi/Downloads/info-patients-v0_4/edf/de...,male
0,00003635,6.0,/home/zozi/Downloads/info-patients-v0_4/edf/de...,male
0,00000795,27.0,/home/zozi/Downloads/info-patients-v0_4/edf/de...,male
0,00003306,13.0,/home/zozi/Downloads/info-patients-v0_4/edf/de...,female
0,00004671,22.0,/home/zozi/Downloads/info-patients-v0_4/edf/de...,gentleman
...,...,...,...,...
0,00008859,29.0,/home/zozi/Downloads/info-patients-v0_4/edf/tr...,woman
0,00008829,48.0,/home/zozi/Downloads/info-patients-v0_4/edf/tr...,female
0,00008835,21.0,/home/zozi/Downloads/info-patients-v0_4/edf/tr...,female
0,00008886,81.0,/home/zozi/Downloads/info-patients-v0_4/edf/tr...,female


In [268]:
clean_df.to_csv('../to_csv/age/age_gender_by_patient.csv', index=False)

In [245]:
clean_df[clean_df.Genre.isnull()]

,Patient,Age,path,Genre
0,00008174,91.0,/home/zozi/Downloads/info-patients-v0_4/edf/de...,NaN
0,00004208,89.0,/home/zozi/Downloads/info-patients-v0_4/edf/tr...,NaN
0,00000900,8.0,/home/zozi/Downloads/info-patients-v0_4/edf/tr...,NaN
0,00001795,9.0,/home/zozi/Downloads/info-patients-v0_4/edf/tr...,NaN
0,00004719,72.0,/home/zozi/Downloads/info-patients-v0_4/edf/tr...,NaN
0,00002214,6.0,/home/zozi/Downloads/info-patients-v0_4/edf/tr...,NaN
0,00002232,9.0,/home/zozi/Downloads/info-patients-v0_4/edf/tr...,NaN
0,00005180,43.0,/home/zozi/Downloads/info-patients-v0_4/edf/tr...,NaN
0,00005103,22.0,/home/zozi/Downloads/info-patients-v0_4/edf/tr...,NaN
0,00002126,3.0,/home/zozi/Downloads/info-patients-v0_4/edf/tr...,NaN


In [124]:
len(df.Patient.unique())

556

In [125]:
df.shape

(1165, 4)

In [136]:
df[df.Patient == '00002991']

,Patient,Age,path,Genre
0,00002991,62,/home/zozi/Downloads/info-patients-v0_4/edf/tr...,male
0,00002991,59,/home/zozi/Downloads/info-patients-v0_4/edf/tr...,male
0,00002991,63,/home/zozi/Downloads/info-patients-v0_4/edf/tr...,male
0,00002991,63,/home/zozi/Downloads/info-patients-v0_4/edf/tr...,male
0,00002991,63,/home/zozi/Downloads/info-patients-v0_4/edf/tr...,male


In [128]:
from collections import Counter
z = df.Patient.tolist()
d = Counter(z)

{k: v for k, v in sorted(d.items(), key=lambda item: item[1], reverse=True)}

{'00009578': 30,
 '00006514': 20,
 '00006546': 18,
 '00008295': 13,
 '00008453': 11,
 '00005427': 11,
 '00009623': 11,
 '00009839': 10,
 '00008544': 9,
 '00010062': 9,
 '00008552': 9,
 '00010418': 9,
 '00009158': 9,
 '00002289': 8,
 '00008512': 8,
 '00000883': 8,
 '00007032': 8,
 '00010489': 8,
 '00005479': 7,
 '00009994': 7,
 '00009245': 7,
 '00006440': 7,
 '00006059': 6,
 '00009697': 6,
 '00010022': 6,
 '00010639': 6,
 '00006811': 6,
 '00007235': 6,
 '00000906': 6,
 '00011972': 6,
 '00007936': 6,
 '00008738': 6,
 '00008303': 6,
 '00002348': 6,
 '00006535': 6,
 '00008480': 6,
 '00008476': 6,
 '00004087': 5,
 '00009866': 5,
 '00008889': 5,
 '00006904': 5,
 '00005765': 5,
 '00002991': 5,
 '00002521': 5,
 '00000492': 5,
 '00000254': 5,
 '00006230': 5,
 '00012262': 5,
 '00010480': 5,
 '00012229': 5,
 '00009231': 5,
 '00006455': 5,
 '00010591': 5,
 '00010209': 5,
 '00001027': 4,
 '00004671': 4,
 '00004594': 4,
 '00001981': 4,
 '00000258': 4,
 '00010106': 4,
 '00008460': 4,
 '00008479': 4,
